In [6]:
import sys

sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\open_ai_assistant_v2")

from src.tools.vanna.vanna_tool import VannaTool

vn = VannaTool()

In [5]:
from src.utils.excel_data_reader import get_excel_data

archivo_excel = "../../assets/Diccionario_datos.xlsx"
columnas_hoja_2 = ["ddl", "about_variables", "about_database", "questions", "answers"]
data = get_excel_data(archivo_excel, columnas_hoja_2, "Vanna")

In [23]:
# Para alimentar la db

for ddl in data["ddl"]:
    vn.feed_ddl(ddl=ddl)

for documentation in data["about_variables"]:
    vn.feed_documentation(documentation=documentation)

for documentation in data["about_database"]:
    vn.feed_documentation(documentation=documentation)

for example in data["examples"]:
    q = example["question"]
    sql = example["answer"]
    vn.feed_examples(question=q, sql=sql)

Adding ddl: CREATE TABLE IF NOT EXISTS dbo_v2.fcs_computadores (
    Id INT PRIMARY KEY,
    IP VARCHAR(100),
    Puerto INT,
    Puerto_Secundario INT,
    Id_Modbus INT,
    Id_Modbus_Secundario INT,
    Tag VARCHAR(100),
    Compatibilidad_Modicon VARCHAR(100),
    Id_Computador_Redundante INT,
    Estado VARCHAR(100),
    Master VARCHAR(100),
    Usuario VARCHAR(100),
    Contraseña VARCHAR(100),
    Leer_Tiempo_Real VARCHAR(100),
    Leer_Configuracion VARCHAR(100),
    Leer_Alarmas VARCHAR(100),
    Leer_Eventos VARCHAR(100),
    Leer_Historicos VARCHAR(100),
    Grupo_Destino VARCHAR(100),
    Unidad_Destino VARCHAR(100),
    Grupo_Fuente VARCHAR(100),
    Unidad_Fuente VARCHAR(100),
    Orden_Archivos INT,
    Numero_Maximo_Horarios INT,
    Numero_Maximo_Diarios INT,
    Numero_Maximo_Proves INT,
    Numero_Maximo_Batch INT,
    Tipo_Protocolo VARCHAR(100),
    Tiempo_Proceso_Historico INT,
    Tiempo_Proceso_TiempoReal INT,
    IdFirmware_fk INT,
    IdEquipo_fk INT,
    Serv

In [20]:
# Para borrar todos los datos de la db
vn.remove_all_data()

In [25]:
training_data = vn.get_training_data()
print(training_data)

             id training_data_type  \
0    202540-sql                sql   
1     56159-ddl                ddl   
2     56181-ddl                ddl   
3     56171-ddl                ddl   
4     56211-ddl                ddl   
..          ...                ...   
111   56201-ddl                ddl   
112   56145-ddl                ddl   
113   56165-ddl                ddl   
114   56214-ddl                ddl   
115   56153-ddl                ddl   

                                              question  \
0    find the average value of each variable record...   
1                                                 None   
2                                                 None   
3                                                 None   
4                                                 None   
..                                                 ...   
111                                               None   
112                                               None   
113                  

In [28]:
question = "The last requirement user is asking for is the average temperature for the measurement system with the Tag 546899GGIE"
question = "user is asking for is the average temperature of a computer flow with the Tag 546899GGIE"
generated_sql = vn.generate_sql(question)
print(generated_sql)

SELECT AVG(vd.Valor) AS Average_Temperature
FROM dbo_v2.var_variable_datos vd
JOIN dbo_v2.var_tipo_variable vt ON vd.idVariable_fk = vt.Id
JOIN dbo_v2.fcs_computador_medidor cm ON vd.idSistemaMedicion_fk = cm.Id_Sisema_Medicion
JOIN dbo_v2.fcs_computadores c ON cm.IdComputador_fk = c.Id
WHERE c.Tag = '546899GGIE'
AND vt.Nombre = 'Temperatura (°C)';


In [4]:
res = vn.run_sql(generated_sql)
print(res)

      Id                   Nombre                      Tag
0      1         EMED-3138.11-128         EMED-3138.11-128
1      2         EMED-3138.12-050         EMED-3138.12-050
2      3         EMED-3138.12-052         EMED-3138.12-052
3      4         EMED-3138.12-065         EMED-3138.12-065
4      5         EMED-3138.12-105         EMED-3138.12-105
..   ...                      ...                      ...
145  146      TANQ-3141.03-218_OP      TANQ-3141.03-218_OP
146  147      TANQ-3141.03-206_OP      TANQ-3141.03-206_OP
147  148      TANQ-3138.04-202_OP      TANQ-3138.04-202_OP
148  149  TI-TQ-3138.11-002(Meio)  TI-TQ-3138.11-002(Meio)
149  150                 EMED-117                 EMED-117

[150 rows x 3 columns]


C:\Users\lauth\OneDrive\Desktop\open_ai_assistant_v2\src\tools\vanna\vanna_tool.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, self.conn)
